## Importing libraries and setup llm

In [52]:
import os

In [53]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [54]:
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]="assignment-3"

## Input data

In [55]:
text_data='''
Apple Inc. is an American multinational technology company headquartered in Cupertino,
California, that designs, develops, and sells consumer electronics, computer software, and
online services. It is considered one of the Big Five companies in the U.S. information
technology industry, along with Amazon, Google, Microsoft, and Facebook.
'''

## Chains

### Lower Case Chain

In [56]:
lower_case_template='''
Convert the following text to lowercase.
{text}
'''
lower_case_prompt_template= ChatPromptTemplate.from_template(lower_case_template)

lower_case_chain = lower_case_prompt_template | llm | StrOutputParser()

### Punctuation Removal Chain

In [57]:
rm_punctuation_template='''
Remove all punctuation from the following text
{lower_case_output}
'''
rm_punctuation_prompt_template= ChatPromptTemplate.from_template(rm_punctuation_template)

rm_punctuation_chain = rm_punctuation_prompt_template | llm | StrOutputParser()

### Named Entity Recognition Chain

In [58]:
ner_template='''
Identify and extract named entities from the text. They can be names of people, organizations, locations, etc. The output must be in  JSON format.
{rm_punctuation_output}
'''
ner_prompt_template = ChatPromptTemplate.from_template(ner_template)

ner_chain = ner_prompt_template | llm | StrOutputParser()

### Summarization Chain

In [59]:
summarize_template='''
Summarize the following text in one sentence.
{text}
'''

summarize_prompt_template = ChatPromptTemplate.from_template(summarize_template)

summarize_chain = summarize_prompt_template | llm | StrOutputParser()

## Sequential Chain

In [60]:
sequential_chain = (
    RunnablePassthrough.assign(lower_case_output = lower_case_chain) | 
    RunnablePassthrough.assign(rm_punctuation_output = rm_punctuation_chain) | 
    RunnablePassthrough.assign(ner_output = ner_chain) |
    RunnablePassthrough.assign(summarize_output = summarize_chain))
output = sequential_chain.invoke({"text":text_data})

## Output

In [61]:
for k,v in output.items():
    print(f"{k}: {v}")
    print("\n")
    

text: 
Apple Inc. is an American multinational technology company headquartered in Cupertino,
California, that designs, develops, and sells consumer electronics, computer software, and
online services. It is considered one of the Big Five companies in the U.S. information
technology industry, along with Amazon, Google, Microsoft, and Facebook.



lower_case_output: apple inc. is an american multinational technology company headquartered in cupertino, california, that designs, develops, and sells consumer electronics, computer software, and online services. it is considered one of the big five companies in the u.s. information technology industry, along with amazon, google, microsoft, and facebook.


rm_punctuation_output: apple inc is an american multinational technology company headquartered in cupertino california that designs develops and sells consumer electronics computer software and online services it is considered one of the big five companies in the us information technology i